In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tools import create_silence_recordings, split_train_validation
# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

# and IPython.display for audio output
import IPython.display
import tensorflow as tf

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

from WavDataGenerator import WavDataGenerator
from WavDataGeneratorV2 import WavDataGeneratorV2


from os.path import join as path_join
from os import mkdir
from os.path import isdir, basename
import glob

In [5]:
data_dir = r'C:\Development\kaggle\tensorflow-speech-recognition-challenge\data\train\audio'

noise_folder = path_join(data_dir, '_background_noise_')
silence_folder = path_join(data_dir, 'silence')


In [6]:
if not isdir(silence_folder):
    create_silence_recordings(glob.glob(path_join(noise_folder, '*.wav')), 
                              silence_folder, stride=3000)

In [7]:
labels = ['silence', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

In [8]:
dg = WavDataGeneratorV2(data_dir, labels, nx=128, ny=32)

In [17]:
dataset = tf.data.Dataset.from_generator(dg.generator, 
                                         output_types=(tf.float32, tf.float32), 
                                         output_shapes=(
                                                        tf.TensorShape([dg.nx, dg.ny]),  
                                                        tf.TensorShape([dg.num_labels])
                                                       )
                                         )

In [18]:
NUM_THREADS = 5
dataset = dataset.map(lambda x,y : (x,y), num_parallel_calls=NUM_THREADS).prefetch(buffer_size=256)
dataset = dataset.batch(batch_size=4)
X, y = dataset.make_one_shot_iterator().get_next()

In [23]:
with tf.Session() as sess:
    batch_data, batch_labels = sess.run([X, y])